## 3.1.1 Tấn công sử dụng tính căn bậc e

Bài toán

In [149]:
from Crypto.Util.number import long_to_bytes, bytes_to_long, getPrime, inverse

flag = b'Hello, World!'

p = getPrime(512)
q = getPrime(512)

n = p*q
phi = (p - 1) * (q - 1)
e = 3
d = inverse(e, phi)
c = pow(bytes_to_long(flag), e, n)

print('n =', hex(n))
print('e =', hex(e))
print('c =', hex(c))

n = 0x943006849b3ad8bcb730d7452e6a9f680fc699580debd151f0b9def29e693a585e1a300fd4ae2a07f05bac20d15f5dd60299121006b0fa90700ffce841c069c38c910c263fb2c07256dbbf39cca02bd4e0ae048055080211c1a7610939a73058eaf2602ef0c01cf6e6f1f2ed3a7051124bb3ac3828364be28188960909d3a689
e = 0x3
c = 0x5ca337215068974823c0e00eeefc194fd2734374eacc18e1512c8a91565ea40ed5b53b870b861


Lời giải

In [138]:
# ref: https://gist.github.com/jl4730/47c6c88bef60ac35b9da961dc424fe82
def find_invpow(x: int, n: int):
    # build because the math.log function does not work for super big numbers
    # the method leveraged binary search algorithm
    high = 1
    while high ** n <= x:
        high *= 2
    low = high // 2  # critical to use // instead of / because Python's integers are dynamically sized but not float
    while low < high:
        mid = (low + high) // 2
        if low < mid and mid ** n < x:
            low = mid
        elif high > mid and mid ** n > x:
            high = mid
        else:
            return mid
    return mid + 1

m = find_invpow(c, e)
print(long_to_bytes(m))

b'Hello, World!'


## 3.1.2 Tấn công Hastad Broadcast

Bài toán

In [17]:
from Crypto.Util.number import long_to_bytes, bytes_to_long, getPrime, inverse
from sympy.ntheory.modular import crt
def RSAKeyGen():
    e = 7
    p = getPrime(128)
    q = getPrime(128)
    n = p * q
    phi = (p-1) * (q-1)

    try:
        d = inverse(e, phi)
    except:
        return RSAKeyGen()
    
    return n, e #, d, phi

def enc():
    flag = b'Hello, World!'
    m = bytes_to_long(flag)
    n, e = RSAKeyGen()
    c = pow(m, e, n)

    return c, n, e


Lời giải

In [19]:
e = 7
ns = []
cs = []
for i in range(e):
    c, n, e = enc()
    cs.append(c)
    ns.append(n)

In [20]:
[hex(c) for c in cs]

['0x7b39a6dc1c4da2aba3cfadf5e045fbe31a064f2b44326d8408ca8d53bd1582f8',
 '0x55e7091158f372f9777d337fd65adc015a55eb11f774464ae614aeb55c948f54',
 '0x49aa4adede1cc03a1a2f97fdee38b00c333fcdbad487e3415e920148c99e72a1',
 '0x6bd991da864a592a38e067b0f2212ab15728c5b73869717eae87d86273a7ef5e',
 '0xd5fc5e6c880737ac42490d8fa65e89bacefed236fdbdc6c3dd7cedf554ac8e',
 '0xd3a3d6b0592b2567110305369afa0381bea3a1d8988bdaf6bde73ca0c7b8dc9d',
 '0x67f44953ff33cc2471313b9cc3ca9e53219a6a6e36180736519778d7c3f0b49a']

In [21]:
[hex(n) for n in ns]

['0x996155318d2506b7afa4977ef7a1ff436b73f6a4194ae2306167c3b20c10fa1d',
 '0xbc78b5ccb113fe58749e496b39f8af73aac5e04fc4e6ba416e4ea0d50bc406fb',
 '0x84bb1da6bc942839855240929c4c6d46200c0a63d8b4fbafb6c61b10e13a29f1',
 '0xb1f5dcf1e17ebcd6aceda43c2fcadeff952978874e824b5a94f4215fed2a92b3',
 '0x833118500bc9eaa20698dd670786df5444052c3067b842476dd4c4db552564f5',
 '0xe2a937b7f5dd6a1b68fabba317bf64b9a368af0605ddf9fc3b05a29de02cce1d',
 '0xaf10c56424486718ebd953c46157ab7ff57c13e2a51ad838ecae38499f4f88f3']

In [22]:
me = crt(ns, cs)[0]
print(long_to_bytes(find_invpow(me, e)))

b'Hello, World!'

## 3.2 Tấn công Wiener

Tham khảo: https://github.com/pablocelayes/rsa-wiener-attack.git

Bài toán

In [38]:
import sys
sys.path.append('./rsa-wiener-attack')
import RSAwienerHacker as wa
import RSAvulnerableKeyGenerator as wg

from Crypto.Util.number import long_to_bytes, bytes_to_long

e, n, d = wg.generateKeys(256)

flag = b'Hello, World!'
c = pow(bytes_to_long(flag), e, n)

print('e =', hex(e))
print('n =', hex(n))
print('c =', hex(c))

e = 0xb4189834f2ce48483951d84cf848e3aeb0dfb1b3c12329b50fe0b99121800037
n = 0x2b4f1eb8c9ab9bf9c6bbf8cdf095bdaa49c67e35bf8487c64b9adeaff2f4d851f
c = 0xf6283682e03c5434f3142b9c3e79bda13c6ed3d7802ba96596bd88c1756d31e8


Lời giải

In [40]:
d = wa.hack_RSA(e, n)
print('d =', d)
m = pow(c, d, n)
print(long_to_bytes(m))

Hacked!
d = 7091840790055978951
b'Hello, World!'


## 3.3 Phân tích Fermat

Bài toán

In [57]:
from sympy import nextprime, primefactors
from Crypto.Util.number import getPrime
p = getPrime(2048)
q = nextprime(p)
n = p * q
print(hex(n))

0x86cafa12fab4afeaeb86ff371627e09646af4b1fffd844646ef051488c7e20711cc6982fc58b1b83179d01b826c2c0092515b0d75c6e12c8376555b6e45aa1ac8de02f9d86a90642e133e27f5cc7ce3b64fde0b8f8f4389eb6c04177990c3c121bc2b85056a9f0626d580267db26820f1baf2cd34809e9dda86c6468f4e7437770929ad1f9e8a1f0c60fea3817178576815a93b82a5d882deffe520dfedde06abfc0504f01b2da4a20289fa95877ccfc7e5a40b5aeccaf005b80d528e46d9617c6f94719fd583ba35b95f35e3a08f6f5d15bbfcd2d29e94d7457448c2ab4b9dbda26e204889acaa909dee6faca6f11ff5b945b1af3bb1cf5553cd239a7a6d70b6fe53e3e2f2a51a0ecc7b9d20741fb3e74c0ab48ff7b8dc285e57a13f4a38baa9f3edc5bb4257fb6536504f98dc91d2a793aaf9d9fda793740101116d1583d35aa04ac7c42eca8356e3ba836877ac47d548b3f573e4d64d1ab7dc7e031a54edd2b49804b2b08ef1df656a34ea0b5a8bafe3408ecf0199c3d79bfc5cbb38708bf34b6acebdb8a4e57081dc63d4d1438d8ca962309bb9e729c30b9f2ec075eab081b51aa52c3caebffaa96790e9e35ad5f383f3059d6a3cab94259b31091076db596d8fed3bf17d32b1db71f3d94129967b8bc0533f22029bd05c009d9ff9bde56113199bc03da43674c58f604c41cc8923ee3be

Lời giải

In [61]:
[p, q] = primefactors(n)
print('p =', hex(p))
print('q =', hex(q))

p = 0xb9c2b2820918c70800c8807f773e1003fdbe850d72ca33fcbffc86b0cfe0ec986d4d51b67230f6997c82cd0d7d5d2de0a7a8f520a60e198841063087f63d465f16e88563e002b7899ef12db928006c5c9d9fc4f0b6c3681e7de9261342e927a09d7ee2136c73e681c133e755f92e734fae82fe002f89355f670b569abd504ae4e6913af4a94a5dfcadcf6d876857fe80e52727cc5a79385d525412f6e63f2bd0fb2d1574dbfd150e6c24f55398b780dde9838094261c063e3bb7947811d2799feb58d9d70e3dafc980fb6e44ef9dabcbfd89f6fec2e1af74936d72a9dd698032d6b3031e7cff7a053ae0b0e51c9a428858ecc4f34dc8c90d1b9d84becf6326c1
q = 0xb9c2b2820918c70800c8807f773e1003fdbe850d72ca33fcbffc86b0cfe0ec986d4d51b67230f6997c82cd0d7d5d2de0a7a8f520a60e198841063087f63d465f16e88563e002b7899ef12db928006c5c9d9fc4f0b6c3681e7de9261342e927a09d7ee2136c73e681c133e755f92e734fae82fe002f89355f670b569abd504ae4e6913af4a94a5dfcadcf6d876857fe80e52727cc5a79385d525412f6e63f2bd0fb2d1574dbfd150e6c24f55398b780dde9838094261c063e3bb7947811d2799feb58d9d70e3dafc980fb6e44ef9dabcbfd89f6fec2e1af74936d72a9dd698032d6b3031e7cff7a053ae0b0e51c9